<a href="https://colab.research.google.com/github/Kavit212/WAE-Ensemble/blob/main/WAE_Net_Grad_CAM_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install required software and dependencies
!pip install tensorflow==1.14.0
!pip install keras==2.2.4
!pip install git+https://github.com/Kavit212/SegGradCAM
!pip install h5py==2.10.0

In [ ]:
import numpy as np
import os
import pathlib
import sys
import keras
import datetime 
import json
import collections
import tqdm
import imageio
import scipy
import operator
import pandas as pd
import cv2
from keras import backend as K
import skimage
import csbdeep

In [ ]:
#Version check

print(np.__version__)
#print(os.__version__)
#print(pathlib.__version__)
#print(sys.__version__)
print(keras.__version__)
#print(datetime .__version__)
print(json.__version__)
#print(collections.__version__)
print(tqdm.__version__)
print(imageio.__version__)
print(scipy.__version__)
#print(operator.__version__)
print(pd.__version__)
print(cv2.__version__)
#print(K.__version__)
print(skimage.__version__)
print(csbdeep.__version__)

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!git clone https://github.com/Kavit212/SegGradCAM
%cd SegGradCAM

In [ ]:
!pip install git+https://github.com/qubvel/segmentation_models

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from pathlib import Path
import sys
import matplotlib.pyplot as plt

import segmentation_models as sm

from seggradcam.dataloaders import Cityscapes
#from metrics import iou_coef, dice_coef, dice_loss
from seggradcam.unet import csbd_unet, manual_unet, TrainUnet
from seggradcam.training_write import TrainingParameters, TrainingResults
from seggradcam.training_plots import plot_predict_and_gt, plot_loss, plot_metric
from seggradcam.seggradcam import SegGradCAM, SuperRoI, ClassRoI, PixelRoI, BiasRoI
from seggradcam.visualize_sgc import SegGradCAMplot

In [ ]:
#Resizing images, if needed
#Change number of classes if needed
SIZE_X = 512
SIZE_Y = 512
#Number of classes for segmentation
n_classes = 3

In [ ]:
#Capture training image info as a list
train_images = []

for directory_path in glob.glob("Please enter augmented training images path"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, 1)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images.append(img)
       
#Convert list to array for machine learning processing        
train_images = np.array(train_images)

#Capture mask/label info as a list
train_masks = [] 
for directory_path in glob.glob("Please enter augmented training masks path"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        train_masks.append(mask)
        
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)

In [ ]:
#Capture test image info as a list
test_images = []

for directory_path in glob.glob("Please enter test images path"):
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, 1)       
        #img = cv2.resize(img, (SIZE_Y, SIZE_X))
        test_images.append(img)
       
#Convert list to array for machine learning processing        
test_images = np.array(test_images)

#Capture mask/label info as a list
test_masks = [] 
for directory_path in glob.glob("Please enter test masks path"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        #mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        test_masks.append(mask)
        
#Convert list to array for machine learning processing          
test_masks = np.array(test_masks)

In [ ]:
###############################################
#Encode labels. It is multi dimension array, so we flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

In [ ]:
###############################################
#Encode labels. It is multi dimension array, so we flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = test_masks.shape
test_masks_reshaped = test_masks.reshape(-1,1)
test_masks_reshaped_encoded = labelencoder.fit_transform(test_masks_reshaped)
test_masks_encoded_original_shape = test_masks_reshaped_encoded.reshape(n, h, w)

np.unique(test_masks_encoded_original_shape)

In [ ]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)
test_masks_input = np.expand_dims(test_masks_encoded_original_shape, axis=3)

In [ ]:
X_train = train_images
y_train = train_masks_input
X_test = test_images
y_test = test_masks_input


print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/ unlabeled class

In [ ]:
#from keras.utils import to_categorical
train_masks_cat = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))


test_masks_cat = tf.keras.utils.to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))


In [ ]:
#Reused parameters in all models
#Number of classes and class weight are changed according to the dataset 

n_classes=3
activation='softmax'

LR = 0.0001
optim = keras.optimizers.Adam(LR)


# set class weights for dice_loss according to number of classes 
#dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25, 0.25, 0.25, 0.25]))
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.33, 0.33, 0.33]))  
#dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.2, 0.2, 0.2, 0.2, 0.2]))
#dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)



metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
## Calbacks
# If required, please use one CSV log path for each of the models

csv_logger1 = tf.keras.callbacks.CSVLogger('Please enter your intended callback path', append=False, separator=',')
csv_logger2 = tf.keras.callbacks.CSVLogger('Please enter your intended callback path', append=False, separator=',')
csv_logger3 = tf.keras.callbacks.CSVLogger'Please enter your intended callback path', append=False, separator=',')
csv_logger4 = tf.keras.callbacks.CSVLogger('Please enter your intended callback path', append=False, separator=',')
csv_logger5 = tf.keras.callbacks.CSVLogger('Please enter your intended callback path', append=False, separator=',')

# Other callbacks used for all models
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=10, mode='min', verbose=0)
earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')

In [ ]:
########################################################################
###Model 1
BACKBONE1 = 'resnet34'

preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(X_train)
X_test1 = preprocess_input1(X_test)

# define model
model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model1.compile(optim, total_loss, metrics=metrics)


#print(model1.summary())

###Only train model if pre-trained networks are not available. 
###If pre-trained model is available, please load the model without any training

#history1=model1.fit(X_train1, 
          #y_train_cat,
          #batch_size=8, 
          #epochs=500,
         #verbose=1,
          #callbacks=[earlyStopping, reduce_lr, csv_logger1],
          #validation_data=(X_test1, y_test_cat))


#model1.save('Please enter intended path for saved model')

In [ ]:
model.load_weights('Please enter your pre-trained weight path')

In [ ]:
# This is the last convolutional layer before output
# We recommend users to also try other layers according to your preference

prop_from_layer = model.layers[-1].name
prop_to_layer = 'decoder_stage4b_relu'


In [ ]:
# class 0 = background
# class 1 = cell membrane
# class 2 = nucleus /chromosomes
# class 3 = mitochondria

cls = 0 

import cv2
  
# path
path = r'Please enter test image path'
  
# Using cv2.imread() method
# Using 0 to read image in grayscale mode
img = cv2.imread(path, 1)

#Remove hash when normally processing
#test_img_number = 1
#test_img = X_test[test_img_number]

plt.figure(figsize=(12, 8))
plt.title('Testing Image')
#plt.imshow(test_img[:,:,0], cmap='gray')

clsroi = ClassRoI(model=model,image=image,cls=cls)
newsgc = SegGradCAM(model, image, cls, prop_to_layer,  prop_from_layer, roi=clsroi,
                 normalize=True, abs_w=False, posit_w=False)

newsgc.SGC()

# create an object with plotting functionality
plotter = SegGradCAMplot(newsgc,model=model,n_classes=n_classes)

# plot explanations on 1 picture
#plotter.explainClass()
plotter.explainClass()
